# Practica 1: Predicción de la producción de energía eólica con SCIKIT-LEARN
## Parte 2: Clasificación

### Procesamiento de datos
En primer lugar cargamos los datos y aplicamos el mismo tratamiento que en la primera parte.

In [1]:
import pandas as pd
import numpy as np

In [2]:
wind_ava = pd.read_csv('dataset/wind_ava.csv.gz', compression="gzip")

In [3]:
wind_ava["datetime"] = pd.to_datetime(wind_ava["datetime"])
wind_ava.set_index("datetime", inplace=True)
for c in wind_ava.columns:
  if not c.endswith(".13") and c != "energy":
    wind_ava.drop(c, axis=1, inplace=True)

A continuación, añadimos una columna de "class" que tendrá valor 1 si el valor de la energía se encuentra por encima del tercer cuartil, y 0 si es inferior. Consideraremos 1 como clase "alta" y 0 como clase "baja".

In [4]:
limit = wind_ava["energy"].quantile(q=0.75)
wind_ava["class"] = wind_ava.apply(lambda row: 1 if row["energy"]>=limit else 0,axis=1)

### Evaluación modelo de regresión para valores altos y bajos
A continuación, vamos a entrenar el modelo escogido en la Parte 1, con los datos del conjunto de train. Para los subconjuntos de train y test usaremos los mismos que en la anterior sección. Tras entrenar el modelo, haremos una predicción para los valores de test. Estas predicciones las dividimos en altas y bajas y comparamos su RMSE por separado

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error

In [12]:
test_size = len([0 for x in wind_ava.index if x.year >=2009])
train, test = train_test_split(wind_ava, test_size=test_size, shuffle=False)
X_train = train.drop(['energy', 'class'], axis='columns')
y_train = train[['energy']]
X_test = test.drop(['energy', 'class'], axis='columns')
y_test = test[['energy']]

In [13]:
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('svm', SVR(kernel="rbf", C=900, epsilon=2.25))]
)
pipe.fit(X_train, np.ravel(y_train))
predictions =  pipe.predict(X_test)
y_test.insert(0,"predictions", predictions)
y_high = y_test[y_test["energy"]>=limit]
y_low = y_test[y_test["energy"]<limit]

In [14]:
rmse = root_mean_squared_error(y_test["energy"], y_test["predictions"])
rmse_high = root_mean_squared_error(y_high["energy"], y_high["predictions"])
rmse_low = root_mean_squared_error(y_low["energy"], y_low["predictions"])
print("RMSE global:", rmse,"\nRMSE para valores altos:",rmse_high, "\nRMSE para valores bajos:", rmse_low)

RMSE global: 386.7961147673092 
RMSE para valores altos: 597.7465977875753 
RMSE para valores bajos: 289.89758524036023


Podemos observar como las predicciones para valores altos tienen un mayor error frente a los valores bajos que se encuentran por debajo de la media, que coincide con el rendimiento esperado en la Parte 1. 

### Problema de clasificación

A continuación vamos a evaluar el rendimiento de diferentes clasificadores en este nuevo problema de regresión. Usaremos los conjuntos de train y test definidos anteriormente y con las clases generadas en la primera sección de este programa. Los clasificadores a comparar son los siguientes:   
* Clasificador Dummy: Este clasificador unicamente devuelve la clase mayoritaria. Como tenemos dos clases bastante desbalanceadas, la precisión es probable que se situe en torno al 0.75 pero la precision balanceada por clase será siempre 0.5.
* Árbol de decision: Para este clasificador usaremos dos versiones, para comparar los resultados usando diferentes pesos de clases para balancear nuestros datos. Es de esperar que las clases balanceadas den un mejor rendimiento.
* Vecinos más cercanos: Los datos usados en estos en este clasificador serán escalados previamente con StandardScaler(), elegido en la parte 1, ya que es muy sensible a las unidades.
* Naive Bayes: Este clasificador probabilistico basado en el Teorema de Bayes tiene la ventaja de ser muy rapido haciendo predicciones frente a KNN que tiene que calcular un gran número de distancias.
   
Las métricas de evaluación que usaremos son:
* Accuracy: La métrica más comun y simple, aunque la información que aporta es insuficiente en casos con clases desbalanceadas, ya que incluso los clasificadores Dummy obtienen buenos resultados pero no evalua el rendimiento en las clases minoritarias.
* Recall/TPR: Accuracy para una de las clases, en el caso de este modelo, mide la de la clase mayoritaria "baja".
* TNR: Accuracy para la otra clase, "alta".
* Accuracy balanceada (BAC): Esta métrica es la media del TPR y el TNR, y devuelve un valor más representativo que obliga a clasificar de manera correcta ambas clases para obtener un mejor resultado.

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, recall_score, balanced_accuracy_score

classifiers = {
    "Dummy": DummyClassifier(strategy="most_frequent", random_state=454455),
    "Decision Tree (unbalanced)": DecisionTreeClassifier(random_state=454455),
    "Decision Tree (balanced)": DecisionTreeClassifier(class_weight="balanced", random_state=454455),
    "Nearest Neighbors": Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier())
    ]),
    "Naive Bayes":Pipeline([
        ("scaler", StandardScaler()),
        ("nb", GaussianNB())
    ])
}

X_train = train.drop(['energy', 'class'], axis='columns')
y_train = train[['class']]
X_test = test.drop(['energy', 'class'], axis='columns')
y_test = test[['class']]

for name, clf in classifiers.items():
    clf.fit(X_train, np.ravel(y_train))
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, pos_label=0)
    tnr = recall_score(y_test, y_pred)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)

    print(f"{name} Classifier:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall/TPR: {recall:.4f}")
    print(f"TNR: {tnr:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}\n")

Dummy Classifier:
Accuracy: 0.7600
Recall/TPR: 1.0000
TNR: 0.0000
Balanced Accuracy: 0.5000

Decision Tree (unbalanced) Classifier:
Accuracy: 0.7970
Recall/TPR: 0.8600
TNR: 0.5973
Balanced Accuracy: 0.7286

Decision Tree (balanced) Classifier:
Accuracy: 0.8143
Recall/TPR: 0.8843
TNR: 0.5928
Balanced Accuracy: 0.7385

Nearest Neighbors Classifier:
Accuracy: 0.8350
Recall/TPR: 0.9286
TNR: 0.5385
Balanced Accuracy: 0.7335

Naive Bayes Classifier:
Accuracy: 0.7362
Recall/TPR: 0.7557
TNR: 0.6742
Balanced Accuracy: 0.7150



Para obtener conclusiones sobre este nuevo problema, vamos a fijarnos en la balanced accuracy, que tiene en cuenta el desbalanceo de las clases. Además, deberemos comparar los resultados de los modelos con el modelo dummy para asegurarnos de que el modelo es positivo.
El modelo que presenta mayor balanced accuracy es de nuevo KNN, que además es el que tiene una tasa de recall más alta. Esta medida nos indica cuantos valores positivos son correctamente clasificados. 
También nos gustaría señalar la diferencia entre el método de árboles de decisión balanceados y no balanceados, habiendo arrojado el primero mejores resultados. 
Por útlimo, nos gustaría mencionar la importancia de utilizar la balanced accuracy en lugar de la accuracy normal. Por ejemplo, si no tenemos en cuenta este desbalanceo, el clasificador basado en naive Bayes podría parecer peor que el modelo dummy si nos fijamos en la accuracy normal. Sin embargo, si tenemos en cuenta el desbalanceo de las clases, vemos que en realidad presenta mejores resultados el primero. 